In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tsfresh.transformers import RelevantFeatureAugmenter

In [2]:
# load data

X = pd.read_csv("occupancy.csv", parse_dates=["date"])

y = pd.read_csv("occupancy_target.csv", index_col="id")["occupancy"]

In [3]:
tmp = pd.DataFrame(index=y.index)

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(tmp, y, random_state=0)

In [4]:
kind_to_fc_parameters = {
    "light": {
        "c3": [{"lag": 3}, {"lag": 2}, {"lag": 1}],
        "abs_energy": None,
        "sum_values": None,
        "fft_coefficient": [{"attr": "real", "coeff": 0}, {"attr": "abs", "coeff": 0}],
        "spkt_welch_density": [{"coeff": 2}, {"coeff": 5}, {"coeff": 8}],
        "agg_linear_trend": [
            {"attr": "intercept", "chunk_len": 50, "f_agg": "var"},
            {"attr": "slope", "chunk_len": 50, "f_agg": "var"},
        ],
        "change_quantiles": [
            {"f_agg": "var", "isabs": False, "qh": 1.0, "ql": 0.8},
            {"f_agg": "var", "isabs": True, "qh": 1.0, "ql": 0.8},
        ],
    },
    "co2": {
        "fft_coefficient": [{"attr": "real", "coeff": 0}, {"attr": "abs", "coeff": 0}],
        "c3": [{"lag": 3}, {"lag": 2}, {"lag": 1}],
        "sum_values": None,
        "abs_energy": None,
        "sum_of_reoccurring_data_points": None,
        "sum_of_reoccurring_values": None,
    },
    "temperature": {"c3": [{"lag": 1}, {"lag": 2}, {"lag": 3}], "abs_energy": None},
}

In [5]:
augmenter = RelevantFeatureAugmenter(
    column_id="id",
    column_sort="date",
    kind_to_fc_parameters=kind_to_fc_parameters,
)

In [6]:
pipe = Pipeline(
    [
        ("augmenter", augmenter),
        ("classifier", LogisticRegression(random_state=10, C=0.01)),
    ]
)

In [7]:
pipe.set_params(augmenter__timeseries_container=X)

Pipeline(steps=[('augmenter',
                 RelevantFeatureAugmenter(column_id='id', column_sort='date',
                                          kind_to_fc_parameters={'co2': {'abs_energy': None,
                                                                         'c3': [{'lag': 3},
                                                                                {'lag': 2},
                                                                                {'lag': 1}],
                                                                         'fft_coefficient': [{'attr': 'real',
                                                                                              'coeff': 0},
                                                                                             {'attr': 'abs',
                                                                                              'coeff': 0}],
                                                                         'sum_of_reoccurring_data_points': None,
                                                                         'sum_of_reoccurring_values': None,
                                                                         'sum_values': None},
                                                                 'light': {'abs_en...
8097  135 2015-02-10 08:57:00       20.575  33.925000  405.0  516.00   
8098  135 2015-02-10 08:57:59       20.600  33.872500  412.0  521.25   
8099  135 2015-02-10 08:58:59       20.600  33.895000  419.0  525.75   

      humidity_ratio  
0           0.004745  
1           0.004741  
2           0.004739  
3           0.004751  
4           0.004734  
...              ...  
8095        0.005060  
8096        0.005086  
8097        0.005088  
8098        0.005088  
8099        0.005091  

[8100 rows x 7 columns])),
                ('classifier', LogisticRegression(C=0.01, random_state=10))])

In [8]:
pipe.fit(X_train, y_train)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.04it/s]


Pipeline(steps=[('augmenter',
                 RelevantFeatureAugmenter(column_id='id', column_sort='date',
                                          kind_to_fc_parameters={'co2': {'abs_energy': None,
                                                                         'c3': [{'lag': 3},
                                                                                {'lag': 2},
                                                                                {'lag': 1}],
                                                                         'fft_coefficient': [{'attr': 'real',
                                                                                              'coeff': 0},
                                                                                             {'attr': 'abs',
                                                                                              'coeff': 0}],
                                                                         'sum_of_reoccurring_data_points': None,
                                                                         'sum_of_reoccurring_values': None,
                                                                         'sum_values': None},
                                                                 'light': {'abs_en...
8097  135 2015-02-10 08:57:00       20.575  33.925000  405.0  516.00   
8098  135 2015-02-10 08:57:59       20.600  33.872500  412.0  521.25   
8099  135 2015-02-10 08:58:59       20.600  33.895000  419.0  525.75   

      humidity_ratio  
0           0.004745  
1           0.004741  
2           0.004739  
3           0.004751  
4           0.004734  
...              ...  
8095        0.005060  
8096        0.005086  
8097        0.005088  
8098        0.005088  
8099        0.005091  

[8100 rows x 7 columns])),
                ('classifier', LogisticRegression(C=0.01, random_state=10))])

In [9]:
print(classification_report(y_test, pipe.predict(X_test)))

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.08it/s]

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        28
           1       0.86      1.00      0.92         6

    accuracy                           0.97        34
   macro avg       0.93      0.98      0.95        34
weighted avg       0.97      0.97      0.97        34



In [16]:
train_t = pipe[:-1].transform(X_train)

train_t.shape

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.95it/s]


(101, 27)